# Hypergraph Visualization Examples

This notebook demonstrates the `graph.visualize()` method across different graph patterns.

In [ ]:
from hypergraph import END, Graph, ifelse, node, route

## 1. Simple Single Node

The simplest case - one function node.

In [ ]:
@node(output_name="doubled")
def double(x: int) -> int:
    """Double a number."""
    return x * 2


graph = Graph(nodes=[double])
graph.visualize()

## 2. Linear Pipeline

A simple DAG where data flows through multiple nodes in sequence.

In [ ]:
@node(output_name="cleaned")
def clean(raw_text: str) -> str:
    """Clean raw text."""
    return raw_text.strip().lower()


@node(output_name="tokens")
def tokenize(cleaned: str) -> list[str]:
    """Tokenize cleaned text."""
    return cleaned.split()


@node(output_name="embedding")
def embed(tokens: list[str]) -> list[float]:
    """Create embedding from tokens."""
    return [0.1] * len(tokens)


pipeline = Graph(nodes=[clean, tokenize, embed])
pipeline.visualize()

## 3. Multiple Inputs

A node that takes multiple inputs from different sources.

In [ ]:
@node(output_name="query_vec")
def embed_query(query: str) -> list[float]:
    """Embed the query."""
    return [0.1] * 10


@node(output_name="docs")
def retrieve(query_vec: list[float], top_k: int) -> list[str]:
    """Retrieve documents."""
    return ["doc1", "doc2"]


@node(output_name="answer")
def generate(docs: list[str], query: str) -> str:
    """Generate answer from docs and query."""
    return f"Answer based on {len(docs)} docs"


rag = Graph(nodes=[embed_query, retrieve, generate])
rag.visualize()

## 4. With Type Annotations

Use `show_types=True` to display input/output types.

In [ ]:
rag.visualize(show_types=True)

## 5. Bound Inputs

Inputs that have been bound to specific values appear differently.

In [ ]:
# Bind top_k to 5
rag_bound = rag.bind(top_k=5)
rag_bound.visualize(show_types=True)

## 6. Binary Branching with @ifelse

Conditional routing based on a boolean condition.

In [ ]:
@node(output_name="cached_result")
def use_cache(query: str) -> str:
    """Return cached result."""
    return "cached answer"


@node(output_name="fresh_result")
def compute_fresh(query: str) -> str:
    """Compute fresh result."""
    return "fresh answer"


@ifelse(when_true="use_cache", when_false="compute_fresh")
def check_cache(query: str) -> bool:
    """Check if query is in cache."""
    return query in ["hello", "world"]


branching = Graph(nodes=[check_cache, use_cache, compute_fresh])
branching.visualize()

## 7. Multi-way Routing with @route

Route to different nodes based on string return value.

In [ ]:
@node(output_name="small_result")
def process_small(data: str) -> str:
    return "processed small"


@node(output_name="medium_result")
def process_medium(data: str) -> str:
    return "processed medium"


@node(output_name="large_result")
def process_large(data: str) -> str:
    return "processed large"


@route(targets=["process_small", "process_medium", "process_large"])
def classify_size(data: str) -> str:
    """Classify data by size."""
    if len(data) < 10:
        return "process_small"
    elif len(data) < 100:
        return "process_medium"
    return "process_large"


routing = Graph(nodes=[classify_size, process_small, process_medium, process_large])
routing.visualize()

## 8. Cyclic Graph (Agentic Loop)

A graph with cycles for iterative processing. Uses `END` sentinel to break out.

In [ ]:
@node(output_name="response")
def generate_response(messages: list[dict]) -> str:
    """Generate LLM response."""
    return "I can help with that!"


@node(output_name="messages")
def accumulate(messages: list[dict], response: str) -> list[dict]:
    """Add response to message history."""
    return messages + [{"role": "assistant", "content": response}]


@route(targets=["generate_response", END])
def should_continue(messages: list[dict]) -> str:
    """Decide whether to continue the conversation."""
    if len(messages) > 5:
        return END
    return "generate_response"


agent_loop = Graph(nodes=[generate_response, accumulate, should_continue])
agent_loop.visualize()

## 9. Nested Graph (Hierarchical Composition)

Graphs can contain other graphs as nodes. Use `depth` to control expansion.

In [ ]:
# Inner graph: text processing pipeline
@node(output_name="cleaned")
def clean_text(text: str) -> str:
    return text.strip()


@node(output_name="normalized")
def normalize(cleaned: str) -> str:
    return cleaned.lower()


preprocess = Graph(nodes=[clean_text, normalize], name="preprocess")


# Outer graph using the inner graph
@node(output_name="result")
def analyze(normalized: str) -> dict:
    return {"length": len(normalized)}


workflow = Graph(nodes=[preprocess.as_node(), analyze])
print("depth=0 (collapsed):")
workflow.visualize(depth=0)

In [ ]:
print("depth=1 (expanded):")
workflow.visualize(depth=1)

## 10. Multi-level Nesting

Graphs can be nested multiple levels deep.

In [ ]:
# Level 1: Simple transform
@node(output_name="step1_out")
def step1(x: int) -> int:
    return x + 1


@node(output_name="step2_out")
def step2(step1_out: int) -> int:
    return step1_out * 2


inner = Graph(nodes=[step1, step2], name="inner")


# Level 2: Wrap inner + add validation
@node(output_name="validated")
def validate(step2_out: int) -> int:
    return step2_out


middle = Graph(nodes=[inner.as_node(), validate], name="middle")


# Level 3: Wrap middle + add logging
@node(output_name="logged")
def log_result(validated: int) -> int:
    print(f"Result: {validated}")
    return validated


outer = Graph(nodes=[middle.as_node(), log_result])

print("depth=0:")
outer.visualize(depth=0)

In [ ]:
print("depth=1:")
outer.visualize(depth=1)

In [ ]:
print("depth=2 (fully expanded):")
outer.visualize(depth=2)

## 11. Theme Options

Force light or dark theme regardless of environment.

In [ ]:
print("Light theme:")
pipeline.visualize(theme="light")

In [ ]:
print("Dark theme:")
pipeline.visualize(theme="dark")